In [1]:
import json, requests, csv
import pandas as pd
from contextlib import closing
from datetime import timedelta, date, datetime

In [2]:
Batch_ID= 2995585
API_token =  {'Captricity-API-Token': '435c308483e040fe9928418703216584'}
BASE_URL = 'shreddr.captricity.com' 

In [3]:
def get_batch_info(batch_id, headers):
    url = 'https://{}/api/v1/batch/{}'.format(BASE_URL, batch_id)
    batch = requests.get(url, headers=headers)
    return json.loads(batch.text)


 - > get list of child Batches from parent Batch
 

In [4]:
Parent_batch = get_batch_info(Batch_ID,API_token)

In [5]:
Parent_batch

{'content_created': None,
 'children_ids': [2995586, 2995587],
 'reject_reasons': [],
 'is_digitized': True,
 'related_job_id': None,
 'created_with': 'auto',
 'id': 2995585,
 'documents': [],
 'user_id': 31953,
 'last_emailed_by': '',
 'created_by': 'Solution Engineer',
 'parent_id': None,
 'templates': [{'status': 'active',
   'read_only': True,
   'name': 'HCFA Template',
   'created': '2018-11-03T20:09:07.302',
   'pattern': ['and', ['or', 483186, 483187]],
   'page_count': 1,
   'modified': '2018-11-03T20:12:15.978',
   'safe_editable': True,
   'sheets': [{'version_number': None,
     'name': 'HCFA_08_05.png',
     'created': '2018-11-03T20:09:07.697',
     'modified': '2018-11-03T20:09:07.697',
     'template_id': 3158,
     'page_number': -1,
     'image_file': 'd01d11d5-9062-4dcf-ba17-2bddbe7e38dc.png',
     'version_name': None,
     'id': 483187,
     'document_id': None},
    {'version_number': None,
     'name': 'HCFA_02_12.png',
     'created': '2018-11-03T20:09:07.597',


   - > for each child Batch 
        - > get the child Batch info
        - > if it is not the reject Batch
        

In [6]:
def get_children_job_info(parent_batch):
    """
    Returns list of job ids from valid children batches
    """
    job_list = []
    for ids in Parent_batch['children_ids']:
        child_branch = get_batch_info(ids, API_token)
        if not child_branch['reject_reasons']: #verify if it's a reject batch pass if so, else return  jobs id
            job_list.extend(child_branch['job_ids'])
        else:
            pass
            
    return job_list

In [7]:
job_list = get_children_job_info(Parent_batch)
job_list[0]

1920495

        - > pull the associated Job's output CSV
                - > Question- are we always assuming one jobs ID? or should I account for multiple?

In [8]:
# #For use if we want to run mutiple job lists
# def run_jobs(job_list):
#     for job_id in job_list:
#         return get_job_info(job_id, API_token)

In [8]:
def get_job_info(job_id, headers):
    url = 'https://{}/api/v1/job/{}/csv/'.format(BASE_URL, job_id)
    job = requests.get(url, headers=headers)
    
    return job.text

In [9]:
job_text = get_job_info(job_list[0], API_token)

In [10]:
job_text

'name,plan_type,insured_id,patient_name,patient_dob,insured_name,signature,dos_from,dos_to,pos,emg,cpt,modifier,diag_pointer,charges_dollars,charges_cents,day_of_unit,family_plan,npi,dos_from1,dos_to1,pos1,emg1,cpt1,modifier1,diag_pointer1,charges_dollars1,charges_cents1,day_of_unit1,family_plan1,npi1,service_location,billing_phone,billing_info\r\nHW_info_spills_smeared_mc_numbersvv_1-page1.png,CHAMPVA,20340121,jason vorheas,06/20/1979,jason vorheas,kew,05/30/2000,05 30 2001,i,20,401,--blank--,x,300,00,x,4,41260,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,"sap testing oakland, ca",--blank--,"dance payers 1438 bonty circle oakland, ca"\r\nHW_Sample1-page1.png,GROUP HEALTH PLAN,abc012345678,collins brooke,072859,doe john t,rhoncol,04/23/2018,042418,--blank--,--blank--,test 123,7089,--blank--,12500,00,1,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--

job_raw = get_job_info(1920495, API_token) => #403 without token, 400 (Forgot parenthases), 404 now with job id- is it right format? Excellent, Reponse 200

In [12]:
for row in job_text.split():
    print (row)

name,plan_type,insured_id,patient_name,patient_dob,insured_name,signature,dos_from,dos_to,pos,emg,cpt,modifier,diag_pointer,charges_dollars,charges_cents,day_of_unit,family_plan,npi,dos_from1,dos_to1,pos1,emg1,cpt1,modifier1,diag_pointer1,charges_dollars1,charges_cents1,day_of_unit1,family_plan1,npi1,service_location,billing_phone,billing_info
HW_info_spills_smeared_mc_numbersvv_1-page1.png,CHAMPVA,20340121,jason
vorheas,06/20/1979,jason
vorheas,kew,05/30/2000,05
30
2001,i,20,401,--blank--,x,300,00,x,4,41260,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,"sap
testing
oakland,
ca",--blank--,"dance
payers
1438
bonty
circle
oakland,
ca"
HW_Sample1-page1.png,GROUP
HEALTH
PLAN,abc012345678,collins
brooke,072859,doe
john
t,rhoncol,04/23/2018,042418,--blank--,--blank--,test
123,7089,--blank--,12500,00,1,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--

In [13]:
def read_csv_url(job_text):
    reader = csv.reader(job_text.split('\n'), delimiter=',')
    df= pd.DataFrame([row for row in reader])
    df = df.rename(columns=df.iloc[0]).drop(df.index[0])
    return df

In [14]:
df = read_csv_url(job_text)

       - > transform the 3 date column values for all rows
       

In [15]:
df

,name,plan_type,insured_id,patient_name,patient_dob,insured_name,signature,dos_from,dos_to,pos,...,modifier1,diag_pointer1,charges_dollars1,charges_cents1,day_of_unit1,family_plan1,npi1,service_location,billing_phone,billing_info
1,HW_info_spills_smeared_mc_numbersvv_1-page1.png,CHAMPVA,20340121,jason vorheas,06/20/1979,jason vorheas,kew,05/30/2000,05 30 2001,i,...,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,"sap testing oakland, ca",--blank--,"dance payers 1438 bonty circle oakland, ca"
2,HW_Sample1-page1.png,GROUP HEALTH PLAN,abc012345678,collins brooke,072859,doe john t,rhoncol,04/23/2018,042418,--blank--,...,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,"desmoines, ia",(111)112-3456,abc hospital
3,HW_smeared_4-page1.png,CHAMPVA,268994893707,heather dil,--blank--,adele casey,on file,0902055,--blank--,c,...,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,1989 disney ln,(225)8009530,--blank--
4,HW_smeared_10-page1.png,TRICARE,60834561,"tinto, tyler, m.",--blank--,"tinto, tyler, m.",g,08 19 80,09/20/1999,23,...,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,--blank--,"acdc health center michigan, in, 95416",(000)0000000,"doctor's incorporated po box 627 los angels, c..."
5,Printed_Sample1-page1.png,MEDICAID,xx-1234-xx-234,jason bourne,12221978,--blank--,--blank--,02/12/2018,04/12/2018,tx,...,231 xy 12,123,5000000,06,2,--blank--,3456782345,--blank--,--blank--,--blank--
6,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
#Edit 11/12 New solution:


In [14]:
def convert_datetime(df, col_name):

    date_series = df[col_name]
#     for date in date_series:
#         if type(date) is str:
#             date_series.replace(date, pd.to_datetime(date,format='%m%d%Y', errors='ignore'))
#     print(date_series)      
    new_dates = pd.to_datetime(date_series, errors= 'coerce')
    for date in new_dates:
        if not datetime.date:
            pass 
        elif date > datetime.utcnow():
            new_dates.replace(date, date - timedelta(days=365.25*100), inplace=True)
        else:
            pass
    
    return(new_dates)


In [15]:
df['patient_dob'] = convert_datetime(df, 'patient_dob')
# df['dos_from'] = convert_datetime(df, 'dos_from')
# df['dos_to'] = convert_datetime(df, 'dos_to')

In [16]:
df['patient_dob'] #Known issue, not working with row 5 (check if str/int) ()
# df['dos_from']  #Known issue, not working with row 3 (but not sure of the date anyways honestly)
# df['dos_to']

1   1979-06-20
2   1959-07-28
3          NaT
4          NaT
5          NaT
6          NaT
Name: patient_dob, dtype: datetime64[ns]

In [17]:
df['patient_dob']

1   1979-06-20
2   1959-07-28
3          NaT
4          NaT
5          NaT
6          NaT
Name: patient_dob, dtype: datetime64[ns]

In [18]:
pd.to_datetime(df['patient_dob'][5],format='%m%d%Y', errors='coerce')
# df['dos_from'] 
# df['dos_to']

NaT

- > save final C

In [19]:
df.to_csv('my_save_job.csv')